In [14]:
import pandas as pd
from statsmodels.formula.api import ols, wls

Estamos carregando dados usados no artigo "Explicando o dissenso: uma análise empírica do comportamento judicial do Supremo
Tribunal Federal e da Suprema Corte dos Estados Unidos".

O objetivo era entender fatores que eram apontados como motivadores de maior divergência em decisões judiciais a partir de dados do STF e da SCOTUS.

In [24]:
df = pd.read_csv("base_dissenso.csv")

df.head(3)

,court,cohesion,n_votos,carga,stratum,reporter,weight,maj_opin_assigner,year
0,scotus,0.555556,9,119,NaN,NaN,1.0,WHRehnquist,1992.0
1,scotus,1.000000,8,119,NaN,NaN,1.0,WHRehnquist,1992.0
2,scotus,1.000000,9,119,NaN,NaN,1.0,WHRehnquist,1992.0


Vamos separar um DataFrame para cada tribunal:

In [11]:
df_stf = df.query("court == 'STF'")

df_stf.head(3)

,court,cohesion,all_votos,carga,stratum,reporter,weight,maj_opin_assigner,year
1910,STF,0.777778,9,502,1.0,Moreira Alves,5.858318,NaN,1993.0
1911,STF,1.000000,10,457,3.0,Marco Aurelio,2.058824,NaN,1992.0
1912,STF,1.000000,8,616,3.0,Sepulveda Pertence,2.058824,NaN,2001.0


In [12]:
df_scotus = df.query("court == 'scotus'")

df_scotus.head(3)

,court,cohesion,all_votos,carga,stratum,reporter,weight,maj_opin_assigner,year
0,scotus,0.555556,9,119,NaN,NaN,1.0,WHRehnquist,1992.0
1,scotus,1.000000,8,119,NaN,NaN,1.0,WHRehnquist,1992.0
2,scotus,1.000000,9,119,NaN,NaN,1.0,WHRehnquist,1992.0


Vamos criar um modelo que tenta explicar a taxa de coesão nas decisões considerando como variáveis independentes:

- O tribunal que chegou na decisão (`court`)
- O número de processos analisados naquele ano (`carga`)
- O número de ministros que votaram no caso (`n_votos`)

Estamos usando uma função diferente da OLS para criar esse modelo. Você não precisa se preocupar com isso, basta saber que temos também um modelo de regressão linear e podemos interpretar sua tabela da mesma forma que uma tabela da função OLS.

In [ ]:
wls(
    "cohesion ~ court + carga + n_votos",
    weights=df["weight"],
    data=df
).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               cohesion   R-squared:                       0.106
Model:                            WLS   Adj. R-squared:                  0.105
Method:                 Least Squares   F-statistic:                     104.6
Date:                Thu, 08 May 2025   Prob (F-statistic):           5.51e-64
Time:                        13:51:57   Log-Likelihood:                -1280.7
No. Observations:                2643   AIC:                             2569.
Df Residuals:                    2639   BIC:                             2593.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.1600      0.040     28.799      0.000       1.081       1.239
court[T.scotus]    -0.2014      0.013    -15.150      0.000      -0.227      -0.175
carga            1.997e-06   2.53e-06      0.789      0.430   -2.96e-06    6.96e-06
all_votos          -0.0338      0.004     -7.652      0.000      -0.042      -0.025
==============================================================================
Omnibus:                      297.014   Durbin-Watson:                   1.591
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              955.263
Skew:                          -0.565   Prob(JB):                    3.69e-208
Kurtosis:                       5.720   Cond. No.                     2.01e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Queremos falar de um elemento importante nessa tabela. Agora, incluímos aqui uma variável categórica em nosso modelo: a Corte de origem.

Os dois valores possíveis são STF ou SCOTUS. Como isso é representado na regressão e na tabela?

Temos uma linha para essa variável na tabela de resultados.

O que significa o Coeficiente? Por que ele é seguido do texto [T.scotus]?

Outro fator estudado no artigo é a possibilidade do perfil individual de ministros ser um ponto na construção do consenso nas decisões.

Para isso, olhamos para o ministro responsável pela decisão: o relator, no caso do STF.

Vamos construir o modelo apenas para os casos decididos após 2010.

**Importante:** Mesmo restringindo a base de dados, temos uma grande quantidade de ministros nos dados. Vale deixar registrado aqui que o ministro Ayres Britto é o relator presente nos processos a partir de 2010, que não é listado na tabela de resultados do modelo.

O que isso significa?

In [23]:
df_modelo = df_stf.query("year > 2009")

wls(
    'cohesion ~ carga + all_votos + reporter',
    data=df_modelo,
    weights=df_modelo['weight'],
    missing='drop'
).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:               cohesion   R-squared:                       0.620
Model:                            WLS   Adj. R-squared:                  0.423
Method:                 Least Squares   F-statistic:                     3.150
Date:                Thu, 08 May 2025   Prob (F-statistic):            0.00391
Time:                        15:08:36   Log-Likelihood:                -3.0474
No. Observations:                  45   AIC:                             38.09
Df Residuals:                      29   BIC:                             67.00
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
===================================================================================================
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Intercept                           1.0117      0.545      1.857      0.074      -0.103       2.126
reporter[T.Carlos Velloso]          0.2977      0.245      1.217      0.234      -0.203       0.798
reporter[T.Carmen Lucia]            0.3852      0.175      2.198      0.036       0.027       0.743
reporter[T.Cezar Peluso]            0.8658      0.425      2.039      0.051      -0.003       1.734
reporter[T.Dias Toffoli]            0.7488      0.345      2.169      0.038       0.043       1.455
reporter[T.Ellen Gracie]            0.7780      0.390      1.997      0.055      -0.019       1.575
reporter[T.Eros Grau]               0.3078      0.285      1.080      0.289      -0.275       0.891
reporter[T.Gilmar Mendes]           0.4470      0.157      2.852      0.008       0.127       0.768
reporter[T.Joaquim Barbosa]        -0.1381      0.235     -0.588      0.561      -0.618       0.342
reporter[T.Luiz Fux]                0.1086      0.253      0.430      0.671      -0.408       0.625
reporter[T.Marco Aurelio]           0.1621      0.206      0.785      0.439      -0.260       0.584
reporter[T.Mauricio Correa]         0.6253      0.529      1.182      0.247      -0.456       1.707
reporter[T.Ricardo Lewandowski]     0.2072      0.209      0.990      0.331      -0.221       0.635
reporter[T.Teori Zavascki]          0.0997      0.334      0.298      0.768      -0.584       0.783
carga                               0.0004      0.000      1.416      0.167      -0.000       0.001
all_votos                          -0.1527      0.047     -3.221      0.003      -0.250      -0.056
==============================================================================
Omnibus:                        0.126   Durbin-Watson:                   1.682
Prob(Omnibus):                  0.939   Jarque-Bera (JB):                0.328
Skew:                           0.059   Prob(JB):                        0.849
Kurtosis:                       2.599   Cond. No.                     4.85e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.85e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Como interpretar o coeficiente de variáveis categóricas de regressão

O que significa o coeficiente de uma tabela de regressão quando estamos lidando com variáveis categóricas?

Vamos começar do caso mais simples possível: Se tivermos uma variável categórica representada em uma coluna binária, essa variável aparecerá normalmente na tabela. Seu coeficiente também tem o mesmo significado, e apenas temos que ter em mente que seu valor está limitado a ser  0 ou 1.

Por exemplo, imagine que também tivéssemos uma variável chamada sexo, que indica se um ministro é do sexo masculino (1) ou não (0). Se o coeficiente dessa variável for -0,15, isso quer dizer que aumentar essa variável em 1 unidade (ou seja, mudar de 0 para 1) diminui a coesão em 0,15 ponto, mantidas as demais variáveis constantes.

Concretamente, isso indicaria que, em média, os ministros homens têm uma coesão 0,15 ponto menor do que os demais, mantidas as demais variáveis constantes. Nesse caso, a única variação possível é alterar de 0 para 1, é importante termos também isso em mente.

Mas e quando a variável categórica representada como tal? Esse é o caso da variável court, que pode ser STF ou SCOTUS. Esse tipo de variável pode, inclusive, ter mais de duas categorias. Como devemos entender o coeficiente dela?

Nesses casos, o modelo transforma automaticamente nossa variável categórica em variáveis binárias (dummies) para representar as categorias. Ele escolhe uma das categorias como referência (ou base) — essa não aparece na tabela de resultados  — e as demais aparecem na tabela de coeficientes em comparação com essa referência. Assim, o significado do coeficiente é semelhante ao de uma variável binária, mas com uma diferença importante: o coeficiente representa a diferença entre a  categoria indicada como tratamento (por isso o T!) e a categoria de referência.

No nosso exemplo, o modelo escolheu o Ministro Ayres Britto como categoria de referência para relator. Isso significa que:

- Para cada ministro com coeficience significativo (indicado pelo p-valor), o coeficiente indica a diferença de coesão média dos processos relatados pel Ministro Ayres Britto e aquele ministro ou aquela ministra, mantidas as demais variáveis constantes. Outra forma de comunicar é que a diferença média de coesão da ministra Carmen Lúcia em relação ao Ministro Ayres Britto é de 0,385 controlando para a carga de trabalho e número de votos na decisão.

- Para os ministros que não têm coeficiente significativo, não podemos afirmar que há diferença entre a coesão média dos processos relatados por eles e a do Ministro Ayres Britto.

- Esse tipo de resultado de regressão não oferece comparação entre todas as categorias. Para isso, precisaríamos de outros métodos que não vamos abordar. Mas para determinar se uma variável categórica como inteiro tem efeito significativo, precisamos de uma tabela de modelo ANOVA, que iremos abordar nas próximas aulas.

**IMPORTANTE:** Um elemento importante da linguagem para falar de resultados de regressão múltipla é essa ideia de que falamos do coeficiente **mantendo as demais variáveis constantes**, ou ainda **controlando** as demais variáveis. Isso é importante para que possamos entender o efeito de uma variável em relação a outra, e não apenas o efeito dela isoladamente. Essa ideia de controle é fundamental para a análise de regressão.